# Recuperación de pacientes del archivo de XiO
---------

# Procedimiento

- Configurar las herramientas necesarias
- Recuperar mediante FTP los datos archivados de XiO (herramientas externas)
- Subir a **gDrive** de Radiofísica los datos archivados
- Introducir los parámetros relevantes
- Correr los scripts de recuparación
- Descargar los archivos recuperados
- Limpiar en gDrive los archivos creados

## Configuración de herramientas

Instalación de `plastimatch` y `tree`

In [1]:
%%capture
#!sudo apt update

!sudo apt install plastimatch

!sudo apt install tree

Instalación e importación de los módulos `pydicom` y `pyplastimatch`

In [2]:
%%capture

# Instalación de módulos
%pip install pydicom
%pip install pyplastimatch

# Importación de módulos
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast

Cargar los módulos necesrios

In [3]:
# - Acceso a los datos en gDrive
from google.colab import drive
# - Rutas y comprobaciones de existencia de ficheros 
from os import path
from pathlib import Path
from glob import glob
from sys import path as syspath
# - Utilidades de archivo entre gDrive y el sistema local de archivos
from google.colab import files

Montar el disco

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Crear uan referencia al directorio base de trabajo, crearlo e ir a él

In [5]:
%%capture
# Referenciar
baseworkdir = '/content/gdrive/MyDrive/XiO'

# Crear el directorio
!mkdir -p $baseworkdir

# Ir al directorio base
%cd $baseworkdir

Crear la carpeta `scripts` en el directorio base de trabajo en **gDrive** y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [18]:
%%capture
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py


# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

# Comprobar si el plan genérico con un único campo está y si no descargarlo
if not Path('RTPlan.dcm').exists():
  !wget https://raw.githubusercontent.com/csarux/XiOrecovery/main/DICOMSets/RTPlan.dcm

## Subir a **gDrive** de Radiofísica los datos archivados

Arrastrar sobre la carpeta XiO el archivo `.tgz` recuperado

Listar pacientes archivados presentes en el directorio base

In [6]:
!ls -l $baseworkdir/*.tgz

-rw------- 1 root root 20853632 May 31 09:33 /content/gdrive/MyDrive/XiO/584852_CENTENO_GUTIERREZ_ROCIO_Mar_14_2011.tgz
-rw------- 1 root root 68361913 May 25 09:32 /content/gdrive/MyDrive/XiO/592706_GOMEZ_FERNANDEZ_ROSA_MAR_Sep_23_2011.tgz


Definir el fichero que contiene el nombre del paciente archivado

## Introducir los parámetros relevantes

In [7]:
ArchivedPatient = '584852_CENTENO_GUTIERREZ_ROCIO_Mar_14_2011.tgz'  #@param {type: "string"}

Extaer los datos archivados

In [8]:
%%capture
!tar -xvzf $ArchivedPatient

Inferir el nýumero de historia a partir del fichero archivado

In [9]:
PatientID = ArchivedPatient.split('_')[0]
PatientID

'584852'

Recuperar los apellidos y nombre del paciente almacenados en el fichero `demographic` de XiO 

In [10]:
demographicfile = !find ./patient/$PatientID -name demographic -print 
with open(demographicfile[0]) as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]
ApellidosNombre

'CENTENO GUTIERREZ,ROCIO'

Visualizar la estructura de carpetas creadas

In [11]:
!tree -d patient

patient
├── 584852
│   ├── anatomy
│   │   └── studyset
│   │       └── CT1
│   │           ├── DCMData
│   │           └── surfaces
│   ├── dvh
│   └── plan
│       ├── holocraneo
│       └── PLANSETUP
└── 592706
    ├── anatomy
    │   └── studyset
    │       └── CT1
    │           ├── DCMData
    │           ├── ss
    │           └── surfaces
    ├── dvh
    └── plan
        ├── BOOST
        ├── MAMAFSCIZD
        ├── PLANSETUPBOOST
        ├── PLANSETUPMAMA
        └── SUMA

24 directories


Definir las variables con la identificación del estudio CT y el nombre del plan

In [12]:
CT = 'CT1' #@param {type: "string"} 
Plan ='holocraneo' #@param {type: "string"} 

Definir las variables de la prescripción

In [13]:
DosisTotal = 30.0  #@param {type: "number"} 
NumeroSesiones = 10 #@param {type: "integer"} 
DosisSesion =3.0 #@param {type: "number"} 

## Correr los scripts de recuperación

**Estrategia**

- Crear el índice de las imágenes de CT si no existe
- Recuperar las estructuras a partir de los archivos de texto `.msh` 
- Transformar estructuras y dosis a formato DICOM
- Introducir un plan básico genérico y asociarle a la dosis recpuerada y al conjunto de estructuras
- Establecer una marco de referencia coherente entre las imágenes CT y el conjunto plan, dosis, estructuras

####Reacondicionamiento de los archivos de las estructuras

In [14]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
  # Copiar el script para la creación del índice de cortes de CT
  !cp scripts/genindex patient/$PatientID/anatomy/studyset/$CT

  # Cambiar el directorio de trabajo
  %cd patient/$PatientID/anatomy/studyset/$CT

  # Adjudicar permisos de ejecución y crear el índice
  !sudo chmod +x genindex
  !./genindex

  # Crear la carpeta para los archivos máscara con las estructuras
  !mkdir -p ./ss

  # Copiar el script para la creación de las estructuras
  !cp $baseworkdir/scripts/xioss .

  # Correr el script de recuperación de las estructuras
  !sudo chmod +x xioss
  !./xioss --input $baseworkdir/patient/$PatientID --plan $Plan > xioss.log

/content/gdrive/MyDrive/XiO/patient/584852/anatomy/studyset/CT1
mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/externo.mha' and 'ss/externo.mha' are the same file
mv: 'ss/holocraneo.mha' and 'ss/holocraneo.mha' are the same file
mv: 'ss/ptv holocraneo.mha' and 'ss/ptv holocraneo.mha' are the same file
mv: 'ss/ojo dcho.mha' and 'ss/ojo dcho.mha' are the same file
mv: 'ss/ojo izqdo.mha' and 'ss/ojo izqdo.mha' are the same file
mv: 'ss/nerv opt dcho.mha' and 'ss/nerv opt dcho.mha' are the same file
mv: 'ss/nerv opt izqdo.mha' and 'ss/nerv opt izqdo.mha' are the same file
Use of uninitialized value in string eq at ./xioss line 151.
Consistency lack in structure match between XiO contour file and plastimatch prefix output at ./xioss line 151.


####Recuperación del CT, las estructuras y la dosis

In [16]:
# Regresar al directorio base
%cd $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                  'input-prefix' : 'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                  'input-dose-xio' : 'patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                  'output-dicom' : 'xiodcm'} 

# Correr el script de conversión de los datos XiO en estudio DICOM                   
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )


/content/gdrive/MyDrive/XiO

Running 'plastimatch convert' with the specified arguments:
  --input patient/584852/anatomy/studyset/CT1/DCMData
  --input-prefix patient/584852/anatomy/studyset/CT1/ss
  --input-dose-xio patient/584852/plan/holocraneo/dose.1
  --output-dicom xiodcm
... Done.


####Añadir un plan genérico y fijar un marco coherente de referencia

In [19]:
# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!sudo chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico 
!cp RTPlan.dcm xiodcm

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

584852/anatomy/studyset/ doesn't exit at ./deltact line 29.
/content/gdrive/MyDrive/XiO


## Descargar los archivos recuperados

In [ ]:
!zip -rq xiodcm.zip xiodcm
files.download('xiodcm.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Limpiar gDrive

In [ ]:
!rm -rf xiodcm patient xiodcm.zip *.log $ArchivedPatient

In [ ]:
!rm -rf xiodcm patient xiodcm.zip *.log

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

In [ ]:
file_list = gdrive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  if file1['title'] == 'XiO' and file1['mimeType']=='application/vnd.google-apps.folder': file1.Delete()



-------
## Logfiles 

Revisión de *logfiles*

Log del plan

In [ ]:
!cat $PatientID'_xiodcm.log'

Log de estructuras

In [ ]:
!cat patient/$PatientID/anatomy/studyset/$CT/xioss.log